In [1]:
"""
very simple implementation with XGBoost

- Fare, Embarkedの欠損値補完
- SibSp + Parchで家族の人数を計算
- Sex, Embarked, Pclassはダミー変数化
- 扱いが難しそうなTicket, Cabinは破棄
- XGBoostを使用

"""

'\nvery simple implementation with XGBoost\n\n- Fare, Embarkedの欠損値補完\n- SibSp + Parchで家族の人数を計算\n- Sex, Embarked, Pclassはダミー変数化\n- 扱いが難しそうなTicket, Cabinは破棄\n- XGBoostを使用\n\n'

In [2]:
import pandas as pd
import numpy as np

In [3]:
#Load data
train = pd.read_csv("./input/train.csv",header=0)
test = pd.read_csv("./input/test.csv",header=0)

In [4]:
#Fill the missing values 
train['Fare'] = train['Fare'].fillna(train['Fare'].median())
test['Fare'] = test['Fare'].fillna(train['Fare'].median())

In [5]:
train['Embarked'] = train['Embarked'].fillna('S')# S is the most common symbol
test['Embarked'] = test['Embarked'].fillna('S')

In [6]:
#Calc number of family
train['family'] = train['SibSp'] + train['Parch']
test['family'] = test['SibSp'] + test['Parch']

In [7]:
#Convert categorical variable into dummy/indicator variables
def add_dummy(df):
    df['Pclass'] = df['Pclass'].astype(np.str)
    temp = pd.get_dummies(df[['Sex','Embarked','Pclass']], drop_first = False)
    temp['PassengerId'] = df['PassengerId']
    return pd.merge(df, temp)
train = add_dummy(train)
test = add_dummy(test)

In [8]:
#Drop unnecessary feature
def get_feature_mat(df):
    temp = df.drop(columns=['PassengerId','Name','Sex','SibSp','Parch','Ticket','Embarked','Age','Cabin','Pclass'])
    try:
        temp = temp.drop(columns=['Survived'])
    except:
        pass
    print (temp)
    return temp.as_matrix()
x_train = get_feature_mat(train)
x_test = get_feature_mat(test)

         Fare  family  Sex_female  Sex_male  Embarked_C  Embarked_Q  \
0      7.2500       1           0         1           0           0   
1     71.2833       1           1         0           1           0   
2      7.9250       0           1         0           0           0   
3     53.1000       1           1         0           0           0   
4      8.0500       0           0         1           0           0   
5      8.4583       0           0         1           0           1   
6     51.8625       0           0         1           0           0   
7     21.0750       4           0         1           0           0   
8     11.1333       2           1         0           0           0   
9     30.0708       1           1         0           1           0   
10    16.7000       2           1         0           0           0   
11    26.5500       0           1         0           0           0   
12     8.0500       0           0         1           0           0   
13    

/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [9]:
y_train = train['Survived'].as_matrix()


/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [10]:
#Fit and Predict
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100)
xgb.fit(x_train, y_train)
y_test_xgb = xgb.predict(x_test)

In [11]:
#Generate timestamp
from datetime import datetime, timedelta, timezone
JST = timezone(timedelta(hours=+9), 'JST')
ts = datetime.now(JST).strftime('%y%m%d%H%M')

In [12]:
#Save
test["Survived"] = y_test_xgb.astype(np.int)
test[["PassengerId","Survived"]].to_csv(('submit_'+ts+'.csv'),index=False)

In [13]:
test["Survived"].head()


0    0
1    1
2    0
3    0
4    1
Name: Survived, dtype: int64